- required packages
    - numpy 
- Class | MonoPhoneHMM 
    - function | __init__
    - assigns attributes
        - regarding feature
            - .num_dims
        - regarding HMM
            - .phones
            - .num_phones
            - .num_states
            - .trans
            - .elem_prob
            - .state_prob
            - .loglikelihood
            - .trans.accumulatros
            - regarding Viterbi algorithm 
                - .alpha
                - .beta
                - .score
                - .viterbi_score
                - .track
        - regarding GMM
            - .num_mixture
            - .pdf
            - .pdf_accumulatros
        - regarding calculation
            - LZERO
            - LSMALL
            - ZERO
            - MINVAR
            
    - function | make_ptoro
    - create HMM prototype, the initial HMM 
        - variables 
            - phone_list, num_states, prob_loop, num_dims
        - steps
            - assign
                - phones, num_phones, num_states, num_mixture
            - create 
                - pdf for each phone, state, and mixture
                - transition matrix for each phone and state
                
    - function | calc_gconst
    - calculate log of the g constant
        - variables
            - variance
        - steps
            - calculate g constant 
            - D * log(2* pi) + np.sum(np.log(variance))
            
    - function | save_hmm
    - save attribute values, gaussian distribution parameters, transition probability matrices probabilities
        - variables
            - self, filename
        - steps
            - create hmmjson dictionary
            - save num_phones, num_states, num_mixture, num_dims with keys
            - save nested dictionaries with following order: 
                - HMMS = [Phones1, Phones2, Phones3 ..] | Phones1 = {phone, HMM} | 
                    - HMM = [State1, State2, State3 ...] | State1 = {state, trans, GMM} |
                        - GMM = [Mixture1, Mixture2, Mixture3 ...] | Mixture1 = {mixture, weight, mu, variance, gConst|
            - json.dump hmmjson to filename
    - function | load_hmm
    - load attribute values, gaussian distribution parameters, transition probability matrices probabilities
        - variables
            - self, filename
        - steps
            - read json file from filename
            - load num_phones, num_states, num_mixture, num_dims from json file to self
            - load nested transition matrix and pdf parameter matrix

    - function | calc_pdf
    - calculate log likelihood of a frame with pdf[p][s][m]
        - variables
           - self, pdf, obs
               - pdf: not self.pdf, but assigned probability distribution by user
        - steps
            - calculate the value of f(x|é) without gConst
            - reshape the value as (num_frames, 1) 
            - calculate the log likelihood with gConst 
    - function | calc_out_prob
    - calculate the emission probability of a frame given state
        - variables
            - self, feat, label
        - steps 
            - calculate log probability for each pdf[p][s][m] (self.elem_prob)using calc_pdf
            - calculate the emission probability for each state (self.state_prob) using self.elem_prob and GMM weight parameters
          
    - function | calc_alpha
    - calculate the alpha probability for each frame
        - variables
            - self, label
        - steps
            - create self.alpha array initialized as log(0), dimension as (label_len, num_states, feat_len)
            - calculate alpha probability for the cases of (1) loop (2) transitioned from prior state (3) transitioned from prior phone
            
    - function | calc_beta
    - calculate the beta probability for each frame
        - variables
            - self, label
        - steps
            - create self.beta array initialized as log(0), dimension as (label_len, num_states, feat_len)
            - caculate beta probability for the cases of (1) loop (2) transition to next state (3) transition to next phone
    - function | flat_init
    - initialize gmm parameters for pdf in each phone, state and mixture
        - variables
            - self, mean, var
        - steps
            - open pdf[p][s][m] using for loop
            - assign the mean, variance and gConst for each pdf
    - function | reset_accumulators
    - reset accumulators used for updating parameters
        - variables
            - self
        - steps
            - create accumulators for GMM pdf
                - create numerator and denominator storage of pdf parameters for distributions of each mixture, state and phone
            - create accumulators for transition probability matirx
    - function | update_accumulators
    - create numerator and denominator storage of loop and transition probability for transition matrix of each state and phone
        - variables
            - self, feat, label    
        - steps
            - extract the number of labels and the frames
            - calculate lconst(alpha(t-1) * trans * beta(t) - loglikelihood)
            - calculate L (lcont * weight * pdf value)
                - update mean and variance for each pdf accumulators
                - update denominator of weight for each pdf accumulators
            - calculate alphabeta(alpha(t) * beta(t) - loglikelihood)
                - update numerator of weight for each pdf accumulators 
                - update denominator for each trans accumulators 
            - calculate loop and transition probability
                - update numerator for each trans accumulators
    - function | update_parameters
    - function | train
    - function | mixup
    - multiply the number of mixtures in GMM by 2
        - variables
            - self
        - steps
            - for PDF in each p, s and m 
                - copy weight, mean, var and gConst 
                - multiply 0.5 to both the original and copied weight(since it gets *0.5)
                - add 0.2 * std to copied mean and extract 0.2 * std to original mean
                - save new gaussian mixture with copied and edited weight, mean, var and gConst to the according phone and state index
            - multiply the number of mixture by 2
            
            
    - function | viterbi_deoding
    - calculate viterbi_score for each transition and state 
        - variables
            - self, label
        - steps 
            - create score and track property
            - for each frame, label and state, 
                - calculate the probability of transition and that of self-loop
                - choose the one with higher probability and note the track(transition or not) and the probability as a score
            - save the final score as viterbi_score 
    - function | back_track
    - function | recognize
    - recognize a word over a give feature audio sequence
        - variables 
            - self, feat, lexicon
        - steps 
            - create a result list
            - for each lexicon dictionary(word, phoneme sequence, phoneme index sequence), 
                - calculate the state-emission probabiliry using calc_out_prob method 
                - and run viterbi_decoding
            - append the word and its according viterbi_score in result and then sort
            - return the most probable word and the detailed word:viterbi_score pair
    - function | phone_alignment
    - function | log_add

In [ ]:
class MonoPhoneHMM():
    def __init__(self):
        self.num_dims = 1 # the dimensions of a feature 
        
        self.num_mixture = 1 # the number of mixes of GMM
        self.pdf = None # parameters for Single Gaussian Model
        
        self.phones = [] # list of phones 
        self.num_phones = 1 # the number of phones
        self.num_states = 1 # the number of states for each phone
        self.trans = None # transition probability matrix
        
        self.elem_prob = None # log probability for each normal distribution 
        self.state_prob = None # log probability for each state 
        self.loglikelihood = 0 # HMM likelihood
        
        self.alpha = None # forward probability
        self.beta = None # backward probability
        
        self.pdf_accumulators = None # variable for pdf parameter update
        self.trans_accumulators = None # variable for trans parameter update
        self.viterbi_score = 0 # score calculated by viterbi algorithm
        self.score = None # accumulated probability for viterbi algorithm
        self.track = None # viterbi track
        
        self.LZERO = -1E10 # aprroximate log(0) value
        self.LSMALL = -0.5E10 # the low boundary value from which to ignore
        self.ZERO = 1E-100 # approximate 0 
        self.MINVAR = 1E-4 # the value to add for variation flooring
        
    def make_proto(self, 
                   phone_list, 
                   num_states,
                   prob_loop,
                   num_dims):
        # assign phone_list, num_phones, num_states, num_dims, num_mixture
        self.phones = phone_list
        self.num_phones = len(self.phones)
        self.num_states = num_states 
        self.num_dims = num_dims
        self.num_mixture = 1
        
        # create pdf for each p, s, m
        self.pdf = []
        for p in range(self.num_phones):
            tmp_p = []
            for s in range(self.num_states):
                tmp_s = []
                for m in range(self.num_mixture):
                    mu = np.zeros(self.num_dims)
                    var = np.ones(self.num_dims)
                    weight = 1.0
                    gconst = self.calc_gconst(var)
                    gaussian = {'weight': weight, 
                               'mu': mu, 
                               'var': var, 
                               'gConst': gconst}
                    tmp_s.append(gaussian)
                tmp_p.append(tmp_s)
            self.pdf.append(tmp_p)
            
            
            
        # create log probability of transition
        prob_next = 1.0 - prob_loop
        log_prob_next = np.log(prob_next) if prob_next > self.ZERO else self.LZERO
        log_prob_loop = np.log(prob_loop) if prob_loop > self.ZERO else self.LZERO
        
        # create transition matrix for each p, s
        self.trans = []
        for p in range(self.num_phones):
            tmp_p = []
            for s in range(self.num_states):
                tmp_trans = np.array([log_prob_loop, log_prob_next])
                tmp_p.append(tmp_trans)
            self.trans.append(tmp_p)
            
            
    def calc_gconst(self, variance):
        # calculate gconst
        gconst = self.num_dims * np.log(2.0 * np.pi) + np.sum(np.log(variance))
        
        return gconst
    
    def flat_init(self, mean, var):
        # open pdf for each phone, state and mixture
        for p in range(self.num_phones):
            for s in range(self.num_states):
                for m in range(self.num_mixture):
                    pdf = self.pdf[p][s][m]
                    # assign mean, variance and gConst for each pdf
                    pdf['mu'] = mean
                    pdf['var'] = var
                    pdf['gConst'] = self.calc_gconst(var)
    
    def load_hmm(self, filename):
        # open hmm json file
        with open(filename, mode = 'r') as f:
            hmmjson = json.load(f)
            
        # read and load num_ attributes
        self.num_phones = hmmjson['num_phones']
        self.num_states = hmmjson['num_states']
        self.num_mixture = hmmjson['num_mixture']
        self.num_dims = hmmjson['num_dims']
        
        # read and load phones
        self.phones = []
        for p in range(self.num_phones):
            hmms = hmmjson['hmms'][p]
            phone = hmms['phone']
            phone_list.append(phone)
        self.phones = phone_list
        
        # read and load hmm parameters
        self.trans = []
        for p in range(self.num_phones):
            tmp_p = []
            hmms = hmmjson['hmms'][p]
            for s in range(self.num_states):
                hmm = hmms['hmm'][s]
                tmp_trans = np.array(hmm['trans'])
                tmp_trans /= np.sum(tmp_trans)
                for i in [0, 1]:
                    trans[i] = np.log(trans[i]) if trans[i] > self.ZERO else self.LZERO
                tmp_p.append(tmp_trans)
            self.trans.append(tmp_p)
                
                
        # read and load gmm parameters
        self.pdf = []
        for p in range(self.num_phones):
            tmp_p = []
            hmms = hmmjson['hmms'][p]
            for s in range(self.num_states):
                tmp_s = []
                hmm = hmms['hmm'][s]
                for m in range(self.num_mixture):
                    gmm = hmm['gmm'][m]
                    weight = gmm['weight']
                    mu = np.array(gmm['mean'])
                    var = np.array(gmm['variance'])
                    gconst = np.array(gmm['gConst'])
                    gaussian = {'weight': weight, 
                               'mu': mu, 
                               'var': var, 
                               'gConst': gconst}
                    tmp_s.append(gaussian)
                tmp_p.append(tmp_s)
            self.pdf.append(tmp_p)
    
    def save_hmm(self, filename):
    # create empty hmmjson file
    hmmjson = {}
    # save attribute values
    hmmjson['num_phones'] = self.num_phones
    hmmjson['num_states'] = self.num_states
    hmmjson['num_mixture'] = self.num_mixture
    hmmjson['num_dims'] = self.num_dims
    
    # save each phone in hmms
    hmmjson['hmms'] = []
    
    for p, phone in range(self.num_phones):
        model_p = {}
        model_p['phone'] = phone
        model_p['hmm'] = []
        # save state for each phone in hmm
        for s in range(self.num_states):
            model_s = {}
            model_s['state'] = s
            model_s['trans'] = list(np.exp(self.trans[p][s]))
            model_s['gmm'] = []
            # save mixture for each state in phone 
            for m in range(self.num_mixture):
                model_m = {}
                model_m['mixture'] = m
                model_m['weight'] = self.pdf[p][s][m]['weight']
                model_m['mu'] = list(self.pdf[p][s][m]['mu'])
                model_m['variance'] = list(self.pdf[p][s][m]['var'])
                model_m['gConst'] = self.pdf[p][s][m]['gConst']
                model_s['gmm'].append(model_m)
            model_p['hmm'].append(model_s)
        hmmjson['hmms'].append(model_s)
    
    # out as a file
    with open(filename, mode = 'w') as f:
        json.dump(hmmjson, f, indent = 4)
        
        
    def calc_pdf(self, pdf, obs):
        # calculate the value of log(f(x|é)) without gConst
        tmp = (obs - pdf['mu']) ** 2 / pdf['var']
        if np.ndim(tmp) >= 2:
            tmp = np.sum(tmp, 1)
        elif np.ndim(tmp) == 1:
            tmp = np.sum(tmp)
        # calculate the value of log(f(x|é)) with gConst
        logprob = -0.5 * (tmp + pdf['gConst'])
        return log prob
    
    def calc_out_prob(self, feat, label):
        # save the number of labels and features
        feat_len = np.shape(feat)[0]
        label_len = len(label)
        
        # create self.elem_prob matrices
        self.elem_prob = np.zeros((label_len, 
                                  self.num_states, 
                                  self.num_mixture, 
                                  feat_len))
        
        # create self.state_prob matrices
        self.state_prob = np.zeros((label_len, 
                                   self.num_states, 
                                   feat_len))
        
        # calculate self.elem_prob and self.state_prob
        for l, p in enumerate(label):
            for s in range(self.num_states):
                # initialize self.state_prob with log(0)
                self.state_prob[l][s][:] = self.LZERO * np.ones(feat_len)
                for m in range(self.num_mixture):
                    pdf = self.pdf[p][s][m]
                    # calculate self.elem_prob
                    self.elem_prob[l][s][m][:] = self.calc_pdf(pdf, feat)
                    # multiply the weight 
                    tmp_prob = np.log(pdf['weight']) + self.elem_prob[l][s][m][:]
                    # add log probability 
                    for t in range(feat_len):
                        self.state_prob[l][s][t] = self.logadd(self.state_prob[l][s][t], tmp_prob[t])
    
    def calc_alpha(self, label):
        # note the number of labels and the number of feature frames
        (label_len, _, feat_len) = np.shape(self.state_prob)
        # create alpha probability array
        self.alpha = self.LZERO * np.ones((label_len, 
                                         self.num_states, 
                                         feat_len))
        # alpha of the first frame should have state = 0, phone = 0
        self.alpha[0][0][0] = self.state_prob[0][0][0]
        # calculate alpha probability for each frame
        for t in range(1, feat_len):
            for l in range(label_len):
                p = label[l]
                for s in range(self.num_states):
                    # in case of loop
                    self.alpha[l][s][t] = self.alpha[l][s][t-1] + self.trans[p][s][0]
                    # in case of transitioned from prior state
                    if s > 0:
                        tmp = self.alpha[l][s-1][t-1] + self.trans[p][s-1][1]
                        if tmp > self.LSMALL:
                            self.alpha[l][s][t] = self.logadd(self.alpha[l][s][t], tmp)
                    # in case of transitioned from prior phone
                    elif l > 0:
                        prev_p = label[l-1]
                        tmp = self.alpha[l-1][-1][t-1] * self.trans[prev_p][-1][1]
                        if tmp > self.LSMALL:
                            self.alpha[l][s][t] = self.logadd(self.alpha[l][s][t], tmp)
                # multiply(since it's log prob, add) the emission probability
                self.alpha[l][s][t] += self.state_prob[l][s][t]
        
        # the final loglikelihood is identical to the alpha probability of the last frame
        self.loglikelihood = self.alpha[-1][-1][-1]
        
        
    
    def calc_beta(self, label):
        # note the number of labels and the number of feature frames
        (label_len, _, feat_len) = np.shape(self.state_prob)
        # create alpha probability array
        self.beta = self.LZERO * np.ones((label_len, 
                                         self.num_states, 
                                         feat_len))
        # beta of the last frame is 0
        self.beta[-1][-1][-1] = 0.0
        # calculate beta probability for each frame 
        for t in (feat_len - 1)[::-1]:
            for l in range(label_len):
                p = label[l]
                for s in range(self.num_states):
                    # in case of loop
                    self.beta[l][s][t] = self.beta[l][s][t+1] + self.trans[p][s][0] + self.state_prob[l][s][t+1]
                    # in case of transitioning to next state
                    if s < self.num_states = 1:
                        tmp = self.beta[l][s+1][t+1] + self.trans[p][s][1] + self.state_prob[l][s+1][t+1]
                        if tmp > self.LSMALL:
                            self.beta[l][s][t] = self.logadd(self.beta[l][s][t], tmp)
                    # in case of transitioning to next phone
                    elif l < label_len - 1:
                        tmp = self.beta[l+1][0][t+1] + self.tran[p][s][1] + self.state_prob[l+1][0][t+1]
                        if tmp > self.LSMALL:
                            self.beta[l][s][t] + self.logadd(self.beta[l][s][t], tmp) 
                
    def reset_accumulators(self):
        # create accumulators for GMM pdf
        self.pdf_accumulators = []
        # create a numerator and denominator storage of parameters(mean, variance, weight) for distributions of each mixture, state and phone 
        for p in range(self.num_phones):
            tmp_p = []
            for s in range(self.num_states):
                tmp_s = []
                for m in range(self.num_mixture):
                    pdf_stats = {}
                    pdf_stats['weight'] = {'num': self.LZERO, 'den': self.LZERO}
                    pdf_stats['mu'] = {'num': self.LZERO * np.ones(self.num_dims), 
                                      'den': self.LZERO}
                    pdf_stats['var'] = {'num': self.LZERO * np.ones(self.num_dims), 
                                       'den': self.LZERO}
                    tmp_s.append(pdf_stats)
                tmp_p.append(tmp_s)
            self.pdf_accumulators.append(tmp_p)
            
        # create accumulators for transition matrices
        self.trans_accumulators = []
        # create a numerator and denominator storage of loop and transition probability
        for p in range(self.num_phones):
            tmp_p = []
            for s in range(self.num_states):
                trans_stats = {}
                trans_stats['num'] = self.LZERO * np.ones(2)
                trans_stats['den'] = self.LZERO
                tmp_p.append(trans_stats)
            self.trans_accumulators.append(tmp_p)
            
            
    def update_accumulators(self, feat, label):
        # extract the number of the labels and the frames
        (label_len, _, feat_len) = np.shape(self.state_prob)
        
        # calculate lconst(alpha(t-1) * trans * beta(t))
        for t in range(feat_len):
            for l in range(label_len):
                p = label[l]
                for s in range(self.num_states):
                    if t == 0 and l == 0 and s == 0:
                        # the first frame has no alpha(t-1) or trans
                        lconst = 0
                    elif t == 0:
                        continue
                    elif s > 0:
                        # consider loop 
                        lconst = self.alpha[l][s][t-1] + self.trans[p][s][0]
                        # consider transition from prior state
                        tmp = self.alpha[l][s-1][t-1] + self.trans[p][s-1][1]
                        if tmp > self.LSMALL:
                            lconst = self.logadd(lconst, tmp)
                    elif l > 0:
                        prev_p = label[l-1]
                        # consider loop 
                        lconst = self.alpha[l][s][t-1] + self.trans[p][s][0]
                        # consider transition from previous phone
                        tmp = self.alpha[l-1][-1][t-1] + self.trans[prev_p][-1][1]
                        if tmp > self.LSMALL:
                            lconst = self.logadd(lconst, tmp)
                    else:
                        # if t != 0 and s == 0 and l == 0
                        # only loop is possible
                        lconst = self.alpha[l][s][t-1] + self.trans[p][s][0]
                        
                    lconst += self.beta[l][s][t] - self.loglikelihood
                    
                    # calculate L(lconst * weight * pdf value)
                    for m in range(self.num_mixture):
                        pdf = self.pdf[p][s][m]
                        pdf_accum = self.pdf_accumulators[p][s][m]
                        L = lconst + pdf['weight'] _ self.elem_prob[l][s][m][t]
                        
                        pdf_accum['mu']['num'] = np.exp(L) * feat[t]
                        if L > self.LSMALL:
                            pdf_accum['mu']['den'] = self.logadd(pdf_accum['mu']['den'], L)
                        
                        dev = feat[t] - pdf['mu']
                        pdf_accum['var']['num'] = np.exp(L) * (dev ** 2)
                        if L > self.LSMALL: 
                            pdf_accum['var']['den'] = pdf_accum['mu']['den']
                        
                        pdf_accum['weight']['num'] = pdf_accum['mu']['den']
                        
                        
        for t in range(feat_len):
            for l in range(label_len):
                p = label[l]
                for s in range(self.num_states):
                    # calculate alphabeta(alpha(t) * beta(t) - loglikelihood)
                    alphabeta = self.alpha[l][s][t] + self.beta[l][s][t] - self.loglikelihood
                    
                    # update denominator of weight 
                    for m in range(self.num_mixture):
                        pdf_accum = self.pdf_accumulators[p][s][m]
                        if m == 0:
                            if alphabeta > self.LSMALL: 
                                pdf_accum['weight']['den'] = self.logadd(pdf_accum['weight']['den'], alphabeta)
                        else:
                            tmp = self.pdf_accumulators[p][s][0]
                            pdf_accum['weight']['den'] = tmp['weight']['den']
                
                            
                    trans_accum = self.trans_accumulators[p][s]
                    
                    if t < feat_len - 1 and aphabeta > self.LSMALL:
                        # update denominator of trans accumulators
                        trans_accum['den'] = self.logadd(trans_accum['den'], alphabeta)
                        # update numerator of trans accumulators
                    if t == feat_len - 1:
                        continue
                    elif s < self.num_states:
                        tmp = self.alpha[l][s][t] + self.trans[p][s][0] + self.beta[l][s][t+1] + self.state_prob[l][s][t+1] - self.loglikelihood
                        if tmp > self.LSMALL:
                            trans_accum['num'][0] = self.logadd(trans_accum['num'][0], tmp)
                        tmp = self.alpha[l][s][t] + self.trans[p][s][1] + self.beta[l][s+1][t+1] + self.state_prob[l][s+1][t+1] - self.loglikelihood
                        if tmp > self.LSMALL:
                            trans_accum['num'][1] = self.logadd(trans_accum['num'][1], tmp)
                    elif l < label_len - 1:
                        tmp = self.alpha[l][s][t] + self.trans[p][s][0] + self.beta[l][s][t+1] + self.state_prob[l][s][t+1] - self.loglikelihood
                        if tmp > self.LSMALL:
                            trans_accum['num'][0] = self.logadd(trans_accum['num'][0], tmp)
                        tmp = self.alpha[l][s][t] + self.trans[p][s][1] + self.beta[l+1][0][t+1] + self.state_prob[l+1][0][t+1] - self.loglikelihood
                        if tmp > self.LSMALL:
                            trans_accum['num'][1] = self.logadd(trans_accum['num'][1], tmp)
                    else:
                        tmp = self.alpha[l][s][t] + self.trans[p][s][0] + self.beta[l][s][t+1] + self.state_prob[l][s][t+1] - self.loglikelihood
                        if tmp > self.LSMALL:
                            trans_accum['num'][0] = self.logadd(trans_accum['num'][0], tmp)
    def mixup(self):
        # for pdf in each phone and states 
        for p in range(self.num_phones):
            for s in range(self.num_states):
                pdf = self.pdf[p][s]
                for m in range(self.num_mixture):
                    # copy weight, mean, var and gConst
                    # adjust both the copied and original weight and mean
                    weight = pdf[m]['weight']
                    weight *= 0.5
                    pdf[m]['weight'] *= 0.5
                    mu = pdf[m]['mu'].copy()
                    var = pdf[m]['var'].copy()
                    std = np.sqrt(var)
                    
                    mu = mu + 0.2 * std
                    pdf[m]['mu'] = pdf[m]['mu'] - 0.2 * std
                    gconst = pdf[m]['gConst']
                    # save gaussian to pdf to the according phone and state index
                    gaussian = {'weight': weight, 
                               'mu': mu, 
                               'var': var, 
                               'gConst': gconst}
                    pdf.append(gaussian)
        # multily the number of mixture in GMM
        self.num_mixture *= 2 
        
    def viterbi_decoding(self, label):
        (label_len, _, feat_len) = np.shape(self.state_prob)
        
        self.score = self.LZERO * np.ones((label_len, self.num_states, feat_len))
        self.track = np.zeros((label_len, self.num_states, feat_len), dtype = np.int16)
        
        self.score[0][0][0] = self.state_prob[0][0][0]
        # for each frame(t), label(l) as state(s), 
        for t in range(1, feat_len):
            for l in range(label_len):
                p = label[l]
                for s in range(self.num_states):
                    if s > 0:
                        # self-loop 
                        p_loop = self.score[l][s][t-1] + self.trans[p][s][0]
                        # transition
                        p_next = self.score[l][s-1][t-1] + self.trans[p][s-1][1]
                        # decide the score and the track
                        cand = [p_loop, p_next]
                        tran = np.argmax(cand)
                        self.score[l][s][t] = cand[tran]
                        self.track[l][s][t] = tran
                        
                    elif l > 0:
                        prev_p = label[l-1]
                        # self-loop
                        p_loop = self.score[l][s][t-1] + self.trans[p][s][0]
                        # transition
                        p_next = self.score[l-1][-1][t-1] + self.trans[prev_p][-1][1]
                        # decide the score and the track 
                        cand = [p_loop, p_next]
                        tran = np.argmax(cand)
                        self.score[l][s][t] = cand[tran]
                        self.track[l][s][t] = tran
                    else:
                        # self-loop
                        p_loop = self.score[l][s][t-1] + self.trans[p][s][0]
                        self.score[l][s][t] = p_loop
                        self.track[l][s][t] = 0
                        
                    # add state_prob
                    self.score[l][s][t] += self.state_prob[l][s][t]
                    
            self.viterbi_score = self.score[-1][-1][-1]
            
    def recognize(self, feat, lexicon):
        # create result list
        result = []
        # for each lexicon dictionary in lexicon
            # compute state-emission probability using calc_out_prob function 
            # and then run viterbi_decoding
        for lex in lexicon:
            label = lex['ph_int']
            self.calc_out_prob(feat, label)
            self.viterbi_decoding(label)
        
        # append the word and its according viterbi_score in result and then sort
        result.append({'word': lex['word'], 
                      'score': self.viterbi_score})
        # return the most probable word and the detailed word:viterbi_score pair 
        return(result[0]['word'], result)